In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# Read the CSV file
df = pd.read_csv('CSVs/sp500_tickers_full_info.csv')
df.head()

# Extract the 'Ticker' column into a list
tickers = df['Ticker'].tolist()

# Clean up any tickers with periods (e.g., 'BRK.B' to 'BRK-B')
tickers = [ticker.replace('.', '-') for ticker in tickers]


In [55]:
revenue_per_share_list = []
valid_tickers = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        
        # Get total revenue from the income statement
        income_stmt = stock.financials
        total_revenue = income_stmt.loc['Total Revenue'][0]  # Most recent period
        
        # Get shares outstanding
        shares_outstanding = stock.info.get('sharesOutstanding')
        
        # Ensure data is not None or zero to avoid division errors
        if total_revenue is not None and shares_outstanding:
            revenue_per_share = total_revenue / shares_outstanding
            revenue_per_share_list.append(revenue_per_share)
            valid_tickers.append(ticker)
        else:
            print(f"Data unavailable for {ticker}")
    except Exception as e:
        print(f"Could not process {ticker}: {e}")


/var/folders/0d/79zy7nxj1pl2rsqrgy7_wqrw0000gn/T/ipykernel_85158/3871388708.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_revenue = income_stmt.loc['Total Revenue'][0]  # Most recent period
/var/folders/0d/79zy7nxj1pl2rsqrgy7_wqrw0000gn/T/ipykernel_85158/3871388708.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_revenue = income_stmt.loc['Total Revenue'][0]  # Most recent period
/var/folders/0d/79zy7nxj1pl2rsqrgy7_wqrw0000gn/T/ipykernel_85158/3871388708.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

In [106]:
df = pd.DataFrame(revenue_per_share_list)
df.fillna(0)

mean = df.mean()
std = df.std()

z_scores = (df - mean) / std

In [107]:
z_scores

,0
0,-0.133736
1,-0.263143
2,-0.306276
3,-0.270603
4,0.071339
...,...
498,-0.272662
499,-0.296726
500,0.001242
501,-0.240709


In [113]:
# Combine everything into the final DataFrame
if revenue_per_share_list:
    result_df = pd.DataFrame({
        'Ticker': valid_tickers,
        'Revenue_per_Share': revenue_per_share_list,
        'Z_score': z_scores[0]  # Selects the single column for Z_score
    })

    # Sort by Z-score
    result_df = result_df.sort_values(by='Z_score', ascending=False).reset_index(drop=True)
else:
    result_df = pd.DataFrame()

result_df.dropna()

,Ticker,Revenue_per_Share,Z_score
0,NVR,3111.106797,14.137169
1,MCK,2382.465665,10.729089
2,COR,1337.564849,5.841765
3,AZO,1093.820389,4.701697
4,CAH,937.262923,3.969430
...,...,...,...
495,VICI,3.426381,-0.398412
496,DOC,3.118200,-0.399853
497,KIM,2.645676,-0.402064
498,NVDA,2.483571,-0.402822


In [14]:

apple_df = yf.Ticker('AAPL').history(period='1Y')

In [32]:
csv_output = 'CSVs/closingPrices.csv'
prices = pd.read_csv(csv_output)
tickers = prices['Ticker'].unique()

with open(csv_output, 'w') as file:
    pass
write_header = True

for ticker in tickers:
    stock_df = yf.Ticker(ticker).history(period='1Y')
    stock_df['Ticker'] = ticker
    company = stock_df[['Ticker', 'Close']]
    company['Date'] = pd.to_datetime(company.index)
    company.to_csv(csv_output, mode='a', header=write_header, index=False)
    write_header = False
    

/var/folders/lt/h9nxt46j4r7dc2xptjh4w0300000gn/T/ipykernel_64634/2475925211.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company['Date'] = pd.to_datetime(company.index)
/var/folders/lt/h9nxt46j4r7dc2xptjh4w0300000gn/T/ipykernel_64634/2475925211.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company['Date'] = pd.to_datetime(company.index)
/var/folders/lt/h9nxt46j4r7dc2xptjh4w0300000gn/T/ipykernel_64634/2475925211.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f